# 画像内容を生成AIに説明させる

画像データをRAGで扱う場合に画像内容がどのようなものであるかをテキスト化して登録しておくと検索しやすくなる。  
そこで取り扱う画像データ（図表など）を生成AIを活用してどのような内容であるか説明させ、説明文とも画像パスを保存すると  
検索時に画像付きの応答を帰すことが可能となる。

## 対象の画像ファイルパスを指定

In [ ]:
# 画像のパス
path="data/image.png"

## 画像説明処理の実行

In [ ]:
import os
import base64
from dotenv import load_dotenv
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers \
    import StrOutputParser
from langchain_openai import AzureChatOpenAI

load_dotenv()

# 1. 画像をBase64文字列に変換するRunnable
encode_image = RunnableLambda(
    lambda path: base64.b64encode(open(path, "rb").read()).decode("utf-8"))

# 2. ユーザーメッセージを構築するRunnable
#    role と content のリストで、テキスト指示＋画像データを渡す
user_message = RunnableLambda(
    lambda b64: [{
        "role": "user",
        "content": [
            {"type": "text", "text": "添付画像の内容について解説してください。"},
            {
                "type": "image",
                "source_type": "base64",
                "mime_type": "image/png",
                "data": b64,
            },
        ]
    }])
# 3. LLMステップ
llm = AzureChatOpenAI(
    api_version=os.environ.get("AZURE_OPENAI_LLM_DEPLOYMENT_API_VERSION"),
    azure_deployment=os.environ.get("AZURE_OPENAI_LLM_DEPLOYMENT_MODEL")
)


# 4. テキストとして出力
OUTPUT_PARSER = StrOutputParser()

# 5. LCELチェーンの定義
chain = (
    encode_image       # 画像パス → Base64
    | user_message     # Base64 → ユーザーメッセージ辞書
    | llm              # モデル呼び出し
    | OUTPUT_PARSER
)

# 6. 実行
result = chain.invoke(path)

# 7. 結果を取得
print(result)